In [1]:
%reload_ext autoreload
%autoreload 2

import projcore as pc
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, MDS
from scipy.spatial.distance import pdist, squareform
import ast
import random
import seaborn as sns
from gensim.models import Word2Vec
import networkx as nx

pd.set_option('display.max_columns', None)

In [2]:
pc.download_kaggle_datasets()
org_battles_df = pd.read_csv('BattlesStaging_01012021_WL_tagged.csv')
card_list_df = pd.read_csv('CardMasterListSeason18_12082020.csv')
winning_card_list_df = pd.read_csv('Wincons.csv')


File BattlesStaging_01012021_WL_tagged/BattlesStaging_01012021_WL_tagged.csv already exists, skipping download
File CardMasterListSeason18_12082020.csv already exists, skipping download
File Wincons.csv already exists, skipping download
clash-royal-data.csv already exists, skipping download


In [28]:
#battles_df = pc.feature_preprocessing(org_battles_df, winning_card_list_df)
org_battles_df.head(100).to_csv("org_battles_subset.csv", index=False)
print(battles_df['winner.high_win_rate'].mean())

0.24741492819344177


In [ ]:
unique_decks = battles_df["winner.card_set"].unique()
deck_wins = battles_df.groupby("winner.card_set").size()
deck_losses = battles_df.groupby("loser.card_set").size()
deck_total_games = deck_wins.add(deck_losses, fill_value=0)
deck_win_rate = (deck_wins / deck_total_games).fillna(0)  # Win rate as fraction
win_rate_dict = deck_win_rate.to_dict()

# DIFFERENT TASK?
winner_deck_counts = battles_df["winner.card_set"].value_counts()
loser_deck_counts = battles_df["loser.card_set"].value_counts()
deck_counts = winner_deck_counts.add(loser_deck_counts, fill_value=0)

In [ ]:
valid_decks = set(deck_counts[deck_counts >= 25].index)
filtered_battles_df = battles_df[battles_df["winner.card_set"].isin(valid_decks)]
unique_decks = filtered_battles_df["winner.card_set"].unique()
print("Number of unique decks:", len(unique_decks))

In [ ]:

# Compute win rates for each deck
deck_wins = battles_df.groupby("winner.card_set").size()
deck_losses = battles_df.groupby("loser.card_set").size()
deck_total_games = deck_wins.add(deck_losses, fill_value=0)
deck_win_rate = (deck_wins / deck_total_games).fillna(0)  # Win rate as fraction


# Store win rates in a dictionary
win_rate_dict = deck_win_rate.to_dict()

# Construct a Deck Similarity Graph (Graph-based Representation)
G = nx.Graph()

# Add decks as nodes with win rate as an attribute
for deck in unique_decks:
    G.add_node(deck, win_rate=win_rate_dict.get(deck, 0))

# Define a function to compute deck similarity (overlap of at least 5 cards)
def deck_similarity(deck1, deck2, min_overlap=5):
    return len(set(deck1) & set(deck2)) >= min_overlap
print("Number of nodes:", G.number_of_nodes())

# Add edges between similar decks (reducing full pairwise computation)
for i in range(len(unique_decks)):
    for j in range(i + 1, len(unique_decks)):
        if deck_similarity(unique_decks[i], unique_decks[j]):
            G.add_edge(unique_decks[i], unique_decks[j])
print("Number of edges:", G.number_of_edges())
# Train DeepWalk using Word2Vec on random walks from the graph
walks = [list(map(str, nx.single_source_shortest_path_length(G, node, cutoff=10).keys())) for node in G.nodes()]
deepwalk_model = Word2Vec(sentences=walks, vector_size=64, window=5, min_count=1, sg=1, workers=4)

# Extract embeddings for each deck
deck_embeddings = {deck: deepwalk_model.wv[str(deck)] for deck in G.nodes()}
print("Number of embeddings:", len(deck_embeddings))
# Convert embeddings to a DataFrame for visualization
deck_embedding_df = pd.DataFrame(deck_embeddings).T
deck_embedding_df["WinRate"] = [G.nodes[deck]["win_rate"] for deck in deck_embedding_df.index]



